## **Установка**

In [ ]:
pip --version

pip 24.1.2 from /usr/local/lib/python3.12/dist-packages/pip (python 3.12)


In [ ]:
!pip install pyspark py4j

## **Проверка работы установленного pyspark.**

In [ ]:
from pyspark import SparkContext, SparkConf

# Настройка Spark
conf = SparkConf().setAppName("Simple RDD Example").setMaster("local[*]")
sc = SparkContext(conf=conf)

# 1. Создание RDD из списка чисел
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
rdd = sc.parallelize(numbers)

# 2. Трансформации: Фильтрация чётных чисел
even_numbers_rdd = rdd.filter(lambda x: x % 2 == 0)

# 3. Действие: Подсчёт суммы чётных чисел
sum_even_numbers = even_numbers_rdd.sum()

# Вывод результата
print("Чётные числа:", even_numbers_rdd.collect())
print("Сумма чётных чисел:", sum_even_numbers)

a = 5 + 6
print(a)

# Остановка SparkContext
sc.stop()

Чётные числа: [2, 4, 6, 8, 10]
Сумма чётных чисел: 30
11


## **Avro vs ORC vs Parquet**

In [ ]:
# Запись данных в Avro. В colab не работает

from pyspark.sql import SparkSession

# Создание SparkSession
spark = SparkSession.builder.appName("Write Avro Example").getOrCreate()

# Пример данных
data = [("Alice", 25), ("Bob", 30), ("Cathy", 29)]
df = spark.createDataFrame(data, ["Name", "Age"])

# Запись данных в Avro
df.write.format("avro").save("path/to/output/avro")

In [ ]:
# Чтение данных из Avro
df_avro = spark.read.format("avro").load("path/to/output/avro")
df_avro.show()

In [ ]:
# Запись данных в Parquet

from pyspark.sql import SparkSession

# Создание SparkSession
spark = SparkSession.builder.appName("Write Parquet Example").getOrCreate()

# Пример данных
data = [("Alice", 25), ("Bob", 30), ("Cathy", 29)]
df = spark.createDataFrame(data, ["Name", "Age"])

# Запись данных в Parquet
df.write.parquet("path/to/output/parquet")

In [ ]:
# Чтение данных из Parquet
df_parquet = spark.read.parquet("path/to/output/parquet")
df_parquet.show()

+-----+---+
| Name|Age|
+-----+---+
|Alice| 25|
|  Bob| 30|
|Cathy| 29|
+-----+---+



In [ ]:
#Запись данных в ORC

from pyspark.sql import SparkSession

# Создание SparkSession
spark = SparkSession.builder.appName("Write ORC Example").getOrCreate()

# Пример данных
data = [("Alice", 25), ("Bob", 30), ("Cathy", 29)]
df = spark.createDataFrame(data, ["Name", "Age"])

# Запись данных в ORC
df.write.orc("path/to/output/orc")

In [ ]:
# Чтение данных из ORC
df_orc = spark.read.orc("path/to/output/orc")
df_orc.show()

+-----+---+
| Name|Age|
+-----+---+
|Alice| 25|
|  Bob| 30|
|Cathy| 29|
+-----+---+



In [ ]:
spark.stop()

## **Чтение файлов и запись в rdd/dataframe**

Чтение данных из файлов в Apache Spark является одним из наиболее часто выполняемых действий, так как Spark предназначен для работы с большими объемами данных, хранящихся в различных форматах. В Spark вы можете читать данные из различных источников, таких как HDFS, S3, локальная файловая система и другие. Spark поддерживает множество форматов файлов, включая текстовые файлы, CSV, JSON, Parquet, Avro и другие.

**Чтение данных с использованием RDD**

In [ ]:
# Создание текстового файла
text_data = """Hello world
Hello Spark
PySpark is amazing
Spark is fast
Hello PySpark
Data processing with Spark is fun
"""

file_path = "/content/text_file.txt"

with open(file_path, "w", encoding="utf-8") as f:
    f.write(text_data)

print(f"Текстовый файл успешно создан: {file_path}")


Текстовый файл успешно создан: /content/text_file.txt


In [ ]:
# Чтение данных с использованием RDD

from pyspark import SparkContext

# Создание SparkContext
sc = SparkContext("local", "Read Text File Example").getOrCreate()

# Чтение текстового файла
rdd = sc.textFile("/content/text_file.txt")

# Пример обработки данных
words = rdd.flatMap(lambda line: line.split(" "))
word_counts = words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

# Вывод результата
print(word_counts.collect())

[('Hello', 3), ('world', 1), ('Spark', 3), ('PySpark', 2), ('is', 3), ('amazing', 1), ('fast', 1), ('Data', 1), ('processing', 1), ('with', 1), ('fun', 1)]


In [ ]:
sc.stop()

**Чтение данных с использованием DataFrame**

In [ ]:
import csv

# Данные для CSV
data = [
    ["Name", "Age", "City"],  # заголовки
    ["Alice", 25, "Amsterdam"],
    ["Bob", 30, "Rotterdam"],
    ["Charlie", 22, "Utrecht"],
    ["Diana", 28, "The Hague"],
    ["Eve", 35, "Eindhoven"]
]

# Путь к файлу
file_path = "/content/text_file_1.csv"

# Создание CSV-файла
with open(file_path, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(data)

print(f"CSV-файл успешно создан: {file_path}")


CSV-файл успешно создан: /content/text_file_1.csv


In [ ]:
#Чтение данных с использованием DataFrame

from pyspark.sql import SparkSession

# Создание SparkSession
spark = SparkSession.builder.appName("Read CSV Example").getOrCreate()

# Чтение CSV-файла
df = spark.read.csv("/content/text_file_1.csv", header=True, inferSchema=True)

# Печать схемы DataFrame
df.printSchema()

# Показ первых 5 строк
df.show(5)

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- City: string (nullable = true)

+-------+---+---------+
|   Name|Age|     City|
+-------+---+---------+
|  Alice| 25|Amsterdam|
|    Bob| 30|Rotterdam|
|Charlie| 22|  Utrecht|
|  Diana| 28|The Hague|
|    Eve| 35|Eindhoven|
+-------+---+---------+



In [ ]:
spark.stop()

**Пример кода с использованием различных параметров**

In [ ]:
import csv

# Данные для CSV
data = [
    ["Name", "Age", "City"],  # заголовки
    ["Alice", 25, "Amsterdam"],
    ["Bob", 30, "Rotterdam"],
    ["Charlie", 22, "Utrecht"],
    ["Diana", 28, "The Hague"],
    ["Eve", 35, "Eindhoven"]
]

# Создание CSV-файла с разделителем ';'
with open("/content/text_file.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file, delimiter=";")
    writer.writerows(data)

print("CSV-файл успешно создан!")

CSV-файл успешно создан!


In [ ]:
#Пример кода с использованием различных параметров

from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.appName("Read all Example").getOrCreate()

# Создание схемы для CSV-файла
schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Age", IntegerType(), True),
    StructField("City", StringType(), True)
])

# Чтение CSV-файла с явной схемой и указанием разделителя
df_custom = spark.read.csv("/content/text_file.csv", header=True, schema=schema, sep=";")

# Печать схемы DataFrame
df_custom.printSchema()

# Показ первых 5 строк
df_custom.show(5)

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- City: string (nullable = true)

+-------+---+---------+
|   Name|Age|     City|
+-------+---+---------+
|  Alice| 25|Amsterdam|
|    Bob| 30|Rotterdam|
|Charlie| 22|  Utrecht|
|  Diana| 28|The Hague|
|    Eve| 35|Eindhoven|
+-------+---+---------+



In [ ]:
spark.stop()

**Чтение JSON-файла**

In [ ]:
import json

# Данные для JSON-файла
data = [
    {"name": "Alice", "age": 25, "city": "Amsterdam"},
    {"name": "Bob", "age": 35, "city": "Rotterdam"},
    {"name": "Charlie", "age": 40, "city": "Utrecht"},
    {"name": "Diana", "age": 28, "city": "The Hague"},
    {"name": "Eve", "age": 50, "city": "Eindhoven"}
]

# Создание JSON-файла
with open("/content/jsonfile.json", "w", encoding="utf-8") as f:
    for record in data:
        f.write(json.dumps(record) + "\n")  # JSON Lines формат (одна запись на строку)

print("JSON-файл успешно создан!")

JSON-файл успешно создан!


In [ ]:
#Чтение JSON-файла

from pyspark.sql import SparkSession

# Создание SparkSession
spark = SparkSession.builder.appName("Read JSON Example").getOrCreate()

# Чтение JSON-файла
df_json = spark.read.json("/content/jsonfile.json")

# Печать схемы DataFrame
df_json.printSchema()

# Показ первых 5 строк
df_json.show(5)

# Пример обработки данных
df_filtered = df_json.filter(df_json["age"] > 30)
df_filtered.show()

root
 |-- age: long (nullable = true)
 |-- city: string (nullable = true)
 |-- name: string (nullable = true)

+---+---------+-------+
|age|     city|   name|
+---+---------+-------+
| 25|Amsterdam|  Alice|
| 35|Rotterdam|    Bob|
| 40|  Utrecht|Charlie|
| 28|The Hague|  Diana|
| 50|Eindhoven|    Eve|
+---+---------+-------+

+---+---------+-------+
|age|     city|   name|
+---+---------+-------+
| 35|Rotterdam|    Bob|
| 40|  Utrecht|Charlie|
| 50|Eindhoven|    Eve|
+---+---------+-------+



Параметры конфигурации при чтении файлов

**header:** Указывает, содержит ли первый ряд файла имена столбцов (только для CSV).

**inferSchema:** Автоматически определяет типы данных (только для CSV и JSON).

**schema:** Явное указание схемы данных.

**delimiter:** Указывает разделитель для значений (например, для CSV).

## **Параметр для автоматического определения типов данных при чтении CSV-файла в DataFrame**

`inferSchema=True` — это параметр, который говорит Spark определить типы данных автоматически, а не считать всё строками (string).

In [45]:
# без inferSchema

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("InferSchemaExample").getOrCreate()

# Создаём пример CSV
data = """name,age,salary
John,30,5000
Jane,35,6000
Mark,40,7000
"""
with open("/content/people.csv", "w") as f:
    f.write(data)

# Читаем CSV без инференса типов
df_no_schema = spark.read.csv("/content/people.csv", header=True, inferSchema=False)
df_no_schema.printSchema()
df_no_schema.show()


root
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- salary: string (nullable = true)

+----+---+------+
|name|age|salary|
+----+---+------+
|John| 30|  5000|
|Jane| 35|  6000|
|Mark| 40|  7000|
+----+---+------+



In [46]:
# с inferSchema=True
df_with_schema = spark.read.csv("people.csv", header=True, inferSchema=True)
df_with_schema.printSchema()
df_with_schema.show()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)

+----+---+------+
|name|age|salary|
+----+---+------+
|John| 30|  5000|
|Jane| 35|  6000|
|Mark| 40|  7000|
+----+---+------+



## **Spark SQL в PySpark**

**Пример использования DataFrame API**

Предположим, есть JSON файл с информацией о людях. Нужно прочитать эти данные, выполнить различные операции, такие как фильтрация, группировка и агрегация, а затем сохранить результат в формате CSV.

Таким образом, как такового SQL нету. То есть есть SQL-подобные функции, но их предоставляет сам Spark.

In [7]:
people =[
        {"name": "John", "age": 30, "department": "HR"},
        {"name": "Doe", "age": 25, "department": "Finance"},
        {"name": "Jane", "age": 35, "department": "HR"},
        {"name": "Mark", "age": 40, "department": "Finance"},
        {"name": "Smith", "age": 23, "department": "Engineering"}
        ]

In [11]:
type(people)

list

In [26]:
import json

# Данные
people = [
    {"name": "John", "age": 30, "department": "HR"},
    {"name": "Doe", "age": 25, "department": "Finance"},
    {"name": "Jane", "age": 35, "department": "HR"},
    {"name": "Mark", "age": 40, "department": "Finance"},
    {"name": "Smith", "age": 23, "department": "Engineering"}
]

with open("/content/people.json", "w", encoding="utf-8") as f:
    for person in people:
        f.write(json.dumps(person, ensure_ascii=False) + "\n")

In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Создание SparkSession
spark = SparkSession.builder.appName("DataFrameAPIExample").getOrCreate()

# Чтение данных из JSON файла
df = spark.read.json("/content/people.json")

# Фильтрация данных
filtered_df = df.filter(col("age") > 30)

# Группировка и агрегация данных
grouped_df = df.groupBy("department").agg({"age": "avg", "name": "count"}).withColumnRenamed("avg(age)", "avg_age").withColumnRenamed("count(name)", "count")

# Сортировка данных
sorted_df = grouped_df.orderBy(col("count").desc())

# Показ результатов
filtered_df.show()
sorted_df.show()

# Сохранение результирующего DataFrame в CSV файл
sorted_df.write.csv("/content/output.csv", header=True)

+---+----------+----+
|age|department|name|
+---+----------+----+
| 35|        HR|Jane|
| 40|   Finance|Mark|
+---+----------+----+

+-----------+-----+-------+
| department|count|avg_age|
+-----------+-----+-------+
|         HR|    2|   32.5|
|    Finance|    2|   32.5|
|Engineering|    1|   23.0|
+-----------+-----+-------+



In [34]:
spark.stop()

**Как работает "оригинальный SQL" в PySpark.**

**Описание задачи**

Предположим, есть два JSON файла: один с информацией о людях, а другой с информацией о департаментах. Нам нужно прочитать эти данные, зарегистрировать их как временные таблицы, выполнить JOIN-запрос и сохранить результат в формате CSV.



Что мы делаем.

- Создаем объект SparkSession.

- Читаем JSON файлы в DataFrame.

- Регистрируем DataFrame как временные таблицы для выполнения SQL-запросов.

- Выполняем SQL-запрос для соединения таблиц по идентификатору департамента.

- Сохраняем результат в CSV файл.


Чтобы все время не сохранять промежуточные таблицы в hive, чтобы обратится к ним с spark.sql.

Можно использовать `createOrReplaceTempView` - создать временную вьюху

In [29]:
# Пример данных (people.json)

people = [
          {"name": "John", "age": 30, "department_id": 1},
          {"name": "Doe", "age": 25, "department_id": 2},
          {"name": "Jane", "age": 35, "department_id": 1},
          {"name": "Mark", "age": 40, "department_id": 2},
          {"name": "Smith", "age": 23, "department_id": 3}
          ]

#Пример данных (departments.json)

departments = [
              {"id": 1, "department_name": "HR"},
              {"id": 2, "department_name": "Finance"},
              {"id": 3, "department_name": "Engineering"}
              ]

with open("/content/people.json", "w", encoding="utf-8") as f:
    for person in people:
        f.write(json.dumps(person, ensure_ascii=False) + "\n")


with open("/content/departments.json", "w", encoding="utf-8") as f:
    for dep in departments:
        f.write(json.dumps(dep, ensure_ascii=False) + "\n")



In [36]:
from pyspark.sql import SparkSession

# Создание SparkSession
spark = SparkSession.builder.appName("SQLAPIExample").getOrCreate()

# Чтение данных из JSON файлов
people_df = spark.read.json("/content/people.json")
departments_df = spark.read.json("/content/departments.json")

# Регистрация DataFrame как временные вьюхи
people_df.createOrReplaceTempView("people")
departments_df.createOrReplaceTempView("departments")

# Выполнение JOIN-запроса с использованием SQL
join_df = spark.sql("""
SELECT p.name, p.age, d.department_name
FROM people p
JOIN departments d
ON p.department_id = d.id
""")

# Показ результатов
join_df.show()

# Сохранение результирующего DataFrame в CSV файл
join_df.write.csv("/content/output_2.csv", header=True)

+-----+---+---------------+
| name|age|department_name|
+-----+---+---------------+
| John| 30|             HR|
|  Doe| 25|        Finance|
| Jane| 35|             HR|
| Mark| 40|        Finance|
|Smith| 23|    Engineering|
+-----+---+---------------+



**PySpark соединения**


In [37]:
from pyspark.sql import SparkSession

# Создание SparkSession
spark = SparkSession.builder.appName("JoinExamples").getOrCreate()

# Пример данных для DataFrame people
people_data = [
    ("John", 30, 1),
    ("Doe", 25, 2),
    ("Jane", 35, 1),
    ("Mark", 40, 2),
    ("Smith", 23, 3)
]
people_columns = ["name", "age", "department_id"]
people_df = spark.createDataFrame(data=people_data, schema=people_columns)

# Пример данных для DataFrame departments
departments_data = [
    (1, "HR"),
    (2, "Finance"),
    (3, "Engineering"),
    (4, "Marketing")
]
departments_columns = ["id", "department_name"]
departments_df = spark.createDataFrame(data=departments_data, schema=departments_columns)

# Показ данных
people_df.show()
departments_df.show()

+-----+---+-------------+
| name|age|department_id|
+-----+---+-------------+
| John| 30|            1|
|  Doe| 25|            2|
| Jane| 35|            1|
| Mark| 40|            2|
|Smith| 23|            3|
+-----+---+-------------+

+---+---------------+
| id|department_name|
+---+---------------+
|  1|             HR|
|  2|        Finance|
|  3|    Engineering|
|  4|      Marketing|
+---+---------------+



**Внутреннее соединение (Inner Join)**

Внутреннее соединение возвращает только те строки, которые имеют совпадающие значения в обеих таблицах.

In [38]:
# Внутреннее соединение
inner_join_df = people_df.join(departments_df, people_df.department_id == departments_df.id, "inner")
inner_join_df.show()

+-----+---+-------------+---+---------------+
| name|age|department_id| id|department_name|
+-----+---+-------------+---+---------------+
| John| 30|            1|  1|             HR|
| Jane| 35|            1|  1|             HR|
|  Doe| 25|            2|  2|        Finance|
| Mark| 40|            2|  2|        Finance|
|Smith| 23|            3|  3|    Engineering|
+-----+---+-------------+---+---------------+



**Левое внешнее соединение (Left Outer Join)**

Левое внешнее соединение возвращает все строки из левой таблицы и соответствующие строки из правой таблицы. Если соответствия нет, возвращаются NULL значения для столбцов правой таблицы.

In [39]:
# Левое внешнее соединение
left_outer_join_df = people_df.join(departments_df, people_df.department_id == departments_df.id, "left_outer")
left_outer_join_df.show()

+-----+---+-------------+---+---------------+
| name|age|department_id| id|department_name|
+-----+---+-------------+---+---------------+
| John| 30|            1|  1|             HR|
|  Doe| 25|            2|  2|        Finance|
| Jane| 35|            1|  1|             HR|
|Smith| 23|            3|  3|    Engineering|
| Mark| 40|            2|  2|        Finance|
+-----+---+-------------+---+---------------+



**Правое внешнее соединение (Right Outer Join)**

Правое внешнее соединение возвращает все строки из правой таблицы и соответствующие строки из левой таблицы. Если соответствия нет, возвращаются NULL значения для столбцов левой таблицы.

In [40]:
# Правое внешнее соединение
right_outer_join_df = people_df.join(departments_df, people_df.department_id == departments_df.id, "right_outer")
right_outer_join_df.show()

+-----+----+-------------+---+---------------+
| name| age|department_id| id|department_name|
+-----+----+-------------+---+---------------+
| Jane|  35|            1|  1|             HR|
| John|  30|            1|  1|             HR|
| Mark|  40|            2|  2|        Finance|
|  Doe|  25|            2|  2|        Finance|
|Smith|  23|            3|  3|    Engineering|
| NULL|NULL|         NULL|  4|      Marketing|
+-----+----+-------------+---+---------------+



**Полное внешнее соединение (Full Outer Join)**

Полное внешнее соединение возвращает все строки, когда есть совпадение в одной из таблиц. Строки без совпадений в обеих таблицах будут иметь NULL значения для столбцов из другой таблицы.

In [42]:
full_outer_join_df = people_df.join(departments_df, people_df.department_id == departments_df.id, "outer")
full_outer_join_df.show()

+-----+----+-------------+---+---------------+
| name| age|department_id| id|department_name|
+-----+----+-------------+---+---------------+
| John|  30|            1|  1|             HR|
| Jane|  35|            1|  1|             HR|
|  Doe|  25|            2|  2|        Finance|
| Mark|  40|            2|  2|        Finance|
|Smith|  23|            3|  3|    Engineering|
| NULL|NULL|         NULL|  4|      Marketing|
+-----+----+-------------+---+---------------+



**Полное перекрестное соединение (Cross Join)**

Полное перекрестное соединение возвращает декартово произведение строк обеих таблиц.

In [43]:
cross_join_df = people_df.crossJoin(departments_df)

cross_join_df.show()

+-----+---+-------------+---+---------------+
| name|age|department_id| id|department_name|
+-----+---+-------------+---+---------------+
| John| 30|            1|  1|             HR|
| John| 30|            1|  2|        Finance|
|  Doe| 25|            2|  1|             HR|
|  Doe| 25|            2|  2|        Finance|
| John| 30|            1|  3|    Engineering|
| John| 30|            1|  4|      Marketing|
|  Doe| 25|            2|  3|    Engineering|
|  Doe| 25|            2|  4|      Marketing|
| Jane| 35|            1|  1|             HR|
| Jane| 35|            1|  2|        Finance|
| Mark| 40|            2|  1|             HR|
| Mark| 40|            2|  2|        Finance|
|Smith| 23|            3|  1|             HR|
|Smith| 23|            3|  2|        Finance|
| Jane| 35|            1|  3|    Engineering|
| Jane| 35|            1|  4|      Marketing|
| Mark| 40|            2|  3|    Engineering|
| Mark| 40|            2|  4|      Marketing|
|Smith| 23|            3|  3|    E

**Соединение с использованием условия (Join with Condition)**

Соединение с использованием условия позволяет задать более сложные условия соединения.

In [44]:
condition_join_df = people_df.join(departments_df, (people_df.department_id == departments_df.id) & (people_df.age > 30), "inner")
condition_join_df.show()

+----+---+-------------+---+---------------+
|name|age|department_id| id|department_name|
+----+---+-------------+---+---------------+
|Jane| 35|            1|  1|             HR|
|Mark| 40|            2|  2|        Finance|
+----+---+-------------+---+---------------+

